1. Log a new model on MLflow and connect to AzureML
2. Insert ???
3. Deploy logged model as stream endpoint
4. Get AzureML model info from model_id (get endpoint_name from model_id)
5. Call endpoint and make predictions given pd.DataFrame with cols: IMAGE_ID, BLOB_FILEPATH
6. Convert predictions to correct format (m_id, i_id, class_prob, predlabel) --> dic
7. Insert predictions into predictions table

In [4]:
import json
import sys
from tqdm.auto import trange, tqdm
import concurrent.futures
import yaml
import os

import numpy as np
import pandas as pd
import cv2
import imageio

from azure.storage.blob import BlobServiceClient, ContainerClient, BlobClient
from azure.identity import DefaultAzureCredential
from azure.ai.ml import MLClient
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model as AzureMLModel,
    Environment,
    CodeConfiguration,
)
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core import Workspace, Model, Experiment, Run
from azureml.core.model import InferenceConfig

import tensorflow as tf
from tensorflow.keras.models import Model as KerasModel, model_from_json
from tensorflow.keras.optimizers import Adam
from keras.callbacks import Callback

import mlflow
import mlflow.keras

from model_serving import prediction
from model_serving import mlflow_log
with open("./model_serving/config.yaml") as f:
    config = yaml.load(f, Loader=yaml.FullLoader)

#### Log a new model on MLflow and connect to AzureML

In [ ]:
model = # load_model that was trained using sql_utils.get_train_df() ????
ml_client = MLClient.from_config(credential=DefaultAzureCredential())
run_name = 'basemodel' # change each time obv

mlflow_log.log_model(model, ml_client, run_name) 

#### Now insert ???

In [ ]:
models = pd.DataFrame(app_utils.get_models())
model_id = models.m_id.max() + 1
model_name = run_name
data = [{'model_name': model_name, 
         'm_id': model_id}]

data_utils.insert_data('models', data)

#### Deploy logged model as stream endpoint

In [ ]:
# how to add a tag to model to associate with model_id

scoring_file = './model_serving/score.py' # update this
deploy.deploy(model_name, scoring_file)

# maybe
test_deployment.test_deployment(model_name)

#### Get AzureML model info from model_id (get endpoint_name from model_id)

In [8]:
ml_client = MLClient(subscription_id="156ffac2-0545-4d4e-aab3-f89b83635d04",
resource_group="defaultresourcegroup-wus2",
workspace_name="pivot", credential=DefaultAzureCredential())
ws = Workspace.from_config('./model_serving/config.json')

In [9]:
experiment_name = config['experiment_name']
run_name = 'basemodel'

experiment = Experiment(workspace=ws, name=experiment_name)
# run = Run(experiment=experiment, run_id=run_name)

# Get a list of all models in the experiment
# models = Model.list(workspace=ws, experiment_name=experiment_name)
runs = experiment.get_runs()
models = []
for run in runs:
    run_models = run.get_models()
    models.extend(run_models)

AttributeError: 'Run' object has no attribute 'get_models'

In [10]:
experiment

Name,Workspace,Report Page,Docs Page
adt-pivot,pivot,Link to Azure Machine Learning studio,Link to Documentation


In [11]:
# get_runs() returns all runs in reverse chronological order (first is most recent)
for i in experiment.get_runs():
    print(i)

Run(Experiment: adt-pivot,
Id: 3ce8d4e5-6343-4d44-be1e-e9c99b3df968,
Type: None,
Status: Completed)
Run(Experiment: adt-pivot,
Id: 0f5b3a2b-fde9-401b-bdaa-31ed72ecc95c,
Type: None,
Status: Completed)


In [16]:
runs = []
for i in experiment.get_runs():
    runs.append(i)

In [23]:
runs[0].get_metrics()

{}

In [ ]:
for run in ml_client.jobs.list():
    print("Name",run.display_name)
    print("Type",run.type)
    print("Compute",run.compute)
    print("Status",run.status)

In [ ]:
endpoint_name = 

In [ ]:
# write a function
def get_model_info(model_id):
    return endpoint_name #, artifacts maybe

#### Call endpoint and make predictions given pd.DataFrame with cols: IMAGE_ID, BLOB_FILEPATH

In [8]:
# Need to figure out how to get 'azureml-model-deployment' from endpoint_name 
# Used in get_predictions()

In [5]:
# sample input
df = pd.DataFrame({'IMAGE_ID': [1, 2, 3], 
                   'BLOB_FILEPATH': ['D20160524T225721_IFCB107/IFCB107D20160524T225721P00213.png', 
                                     'D20160524T225721_IFCB107/IFCB107D20160524T225721P00575.png', 
                                     'D20160524T225721_IFCB107/IFCB107D20160524T225721P00561.png']})

In [6]:
df

,IMAGE_ID,BLOB_FILEPATH
0,1,D20160524T225721_IFCB107/IFCB107D20160524T2257...
1,2,D20160524T225721_IFCB107/IFCB107D20160524T2257...
2,3,D20160524T225721_IFCB107/IFCB107D20160524T2257...


In [12]:
# endpoint_name = get_model_info(m_id)
scoring_uri = config['scoring_uri'].format(endpoint_name='basemodel-endpoint')
api_key = config['api_key']

In [22]:
df['cloud_urls'] = df.BLOB_FILEPATH.apply(lambda x: config['cloud_url'].format(filepath=x))
df

,IMAGE_ID,BLOB_FILEPATH,cloud_urls
0,1,D20160524T225721_IFCB107/IFCB107D20160524T2257...,https://ifcb.blob.core.windows.net/naames/NAAM...
1,2,D20160524T225721_IFCB107/IFCB107D20160524T2257...,https://ifcb.blob.core.windows.net/naames/NAAM...
2,3,D20160524T225721_IFCB107/IFCB107D20160524T2257...,https://ifcb.blob.core.windows.net/naames/NAAM...


In [29]:
from model_serving import prediction
import importlib
importlib.reload(prediction)

<module 'model_serving.prediction' from '/Users/aditis/Documents/DATA590/PIVOT/PIVOT/model_serving/prediction.py'>

In [30]:
preds = prediction.predict(df, scoring_uri, api_key)
preds

<Response [200]>
[[0.0031898769084364176, 2.0788617355327332e-11, 8.588602213421836e-05, 0.00017344176012557, 8.921755068058701e-08, 0.0032710987143218517, 8.093526048469357e-06, 0.9575332403182983, 0.03550129383802414, 0.00023710746609140188], [0.007550970651209354, 5.126950100020622e-07, 0.00035927206045016646, 0.002601270331069827, 2.5344577352370834e-06, 0.20422795414924622, 0.0010179296368733048, 0.7188515663146973, 0.06120515614748001, 0.004182853270322084], [0.00013524151290766895, 1.200628485520383e-11, 1.257793337572366e-05, 0.00022038634051568806, 2.6521347535890527e-05, 0.00010345028567826375, 1.8474192131634481e-07, 0.390915185213089, 0.00037548429099842906, 0.6082109212875366]]


,i_id,probs
0,1,"[0.0031898769084364176, 2.0788617355327332e-11..."
1,2,"[0.007550970651209354, 5.126950100020622e-07, ..."
2,3,"[0.00013524151290766895, 1.200628485520383e-11..."


#### Convert predictions to correct format (m_id, i_id, class_prob, predlabel) --> dic

In [14]:
classes = pd.DataFrame({'label': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9], 
                        'class': ['Chloro',
                                  'Cilliate',
                                  'Crypto',
                                  'Diatom',
                                  'Dictyo',
                                  'Dinoflagellate',
                                  'Eugleno',
                                  'Other',
                                  'Prymnesio',
                                  np.nan]})
classes

,label,class
0,0,Chloro
1,1,Cilliate
2,2,Crypto
3,3,Diatom
4,4,Dictyo
5,5,Dinoflagellate
6,6,Eugleno
7,7,Other
8,8,Prymnesio
9,9,NaN


In [66]:
# turn this into function 
preds['predlabel'] = preds.probs.apply(lambda x: classes[classes.label == pd.Series(x).idxmax()]['class'].values[0])
#

In [67]:
preds

,i_id,probs,predlabel
0,1,"[0.0031898769084364176, 2.0788617355327332e-11...",Other
1,2,"[0.007550970651209354, 5.126950100020622e-07, ...",Other
2,3,"[0.00013524151290766895, 1.200628485520383e-11...",NaN


#### Insert predictions into predictions table

In [ ]:
data = 
insert_data(predictions, data)

In [30]:
probs = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [28]:
classes['class'].values

array(['Chloro', 'Cilliate', 'Crypto', 'Diatom', 'Dictyo',
       'Dinoflagellate', 'Eugleno', 'Other', 'Prymnesio', nan],
      dtype=object)

In [31]:
{classes['class'].values[i]: probs[i] for i in range(10)}

{'Chloro': 1,
 'Cilliate': 2,
 'Crypto': 3,
 'Diatom': 4,
 'Dictyo': 5,
 'Dinoflagellate': 6,
 'Eugleno': 7,
 'Other': 8,
 'Prymnesio': 9,
 nan: 10}

In [11]:
scoring_uri

'https://endpoint_name.westus2.inference.ml.azure.com/score'

In [31]:
os.getcwd()

'/Users/aditis/Documents/DATA590/PIVOT/PIVOT'

In [ ]:
# execute_stored_procedure('Images_To_Predict.sql')

In [37]:
os.chdir('./model_serving/')

In [38]:
os.getcwd()

'/Users/aditis/Documents/DATA590/PIVOT/PIVOT/model_serving'

In [34]:
from model_utils import get_predictions

ImportError: cannot import name 'get_predictions' from 'model_utils' (/Users/aditis/Documents/DATA590/PIVOT/PIVOT/model_serving/model_utils.py)

In [64]:
def get_predictions(df, scoring_uri=scoring_uri, api_key=api_key):
    """
    df: columns [IMAGE_ID, BLOB_FILEPATH]
    scoring_uri:
    api_key:
    """
    with open("../model_serving/config.yaml") as f:
        config = yaml.load(f, Loader=yaml.FullLoader)
    
    df['cloud_urls'] = df.BLOB_FILEPATH.apply(lambda x: config['cloud_url'].format(filepath=x))
    
    preds = prediction.predict(df, scoring_uri, api_key)
    
    classes = pd.DataFrame({'label': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 
                    'class': ['Chloro',
                                'Cilliate',
                                'Crypto',
                                'Diatom',
                                'Dictyo',
                                'Dinoflagellate',
                                'Eugleno',
                                'Other',
                                'Prymnesio',
                                np.nan]})
    
    preds['probs'] = preds.probs.apply(lambda x: {classes['class'].values[i]: x[i] for i in range(9)})
    
    preds['predlabel'] = preds.probs.apply(lambda x: list(x.keys())[pd.Series(x.values()).idxmax()])
    
    return preds

In [65]:
resp = get_predictions(df)

<Response [200]>
[[0.0031898769084364176, 2.0788617355327332e-11, 8.588602213421836e-05, 0.00017344176012557, 8.921755068058701e-08, 0.0032710987143218517, 8.093526048469357e-06, 0.9575332403182983, 0.03550129383802414, 0.00023710746609140188], [0.007550970651209354, 5.126950100020622e-07, 0.00035927206045016646, 0.002601270331069827, 2.5344577352370834e-06, 0.20422795414924622, 0.0010179296368733048, 0.7188515663146973, 0.06120515614748001, 0.004182853270322084], [0.00013524151290766895, 1.200628485520383e-11, 1.257793337572366e-05, 0.00022038634051568806, 2.6521347535890527e-05, 0.00010345028567826375, 1.8474192131634481e-07, 0.390915185213089, 0.00037548429099842906, 0.6082109212875366]]


In [66]:
resp

,i_id,probs,predlabel
0,1,"{'Chloro': 0.0031898769084364176, 'Cilliate': ...",Other
1,2,"{'Chloro': 0.007550970651209354, 'Cilliate': 5...",Other
2,3,"{'Chloro': 0.00013524151290766895, 'Cilliate':...",Other
